In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import math
import yfinance as yf

In [14]:
    new = yf.download(tickers ='EURUSD=X',start ='2020-01-01',end ='2020-05-05',interval= '15m' )
new

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EURUSD=X: 15m data not available for startTime=1577833200 and endTime=1588629600. The requested range must be within the last 60 days.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [11]:
new = web.DataReader('EURUSD=X', data_source='yahoo', start='2020-01-01', end='2020-05-05') 

new

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-01,1.122838,1.115947,1.122083,1.122083,0,1.122083
2020-01-02,1.122712,1.116682,1.121894,1.122083,0,1.122083
2020-01-03,1.118068,1.112570,1.117081,1.117144,0,1.117144
2020-01-06,1.120825,1.115810,1.116246,1.116196,0,1.116196
2020-01-07,1.119946,1.113487,1.119583,1.119799,0,1.119799
...,...,...,...,...,...,...
2020-04-29,1.096996,1.083565,1.087666,1.087725,0,1.087725
2020-04-30,1.101625,1.093542,1.094475,1.094547,0,1.094547
2020-05-03,1.095782,1.090132,1.095746,1.095963,0,1.095963


In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

In [4]:
new = new.filter(['Close'])
new_dataset = new.values

In [5]:
from tensorflow import keras
model = keras.models.load_model('lstm_model.h5')

In [6]:
test = []
for i in range(60,len(new_dataset)):
  test.append(new_dataset[i-60:i,0])
  
test = np.array(test)
test = np.reshape(test, (test.shape[0],test.shape[1],1))
new_predictions = model.predict(test)


In [7]:
new_predictions.shape

(30, 1)